# Extracting relevant tags and text from all html files

In [1]:
html_files_dir=r'C:\Users\Saranga\Desktop\Devopedia\Work\articleRefs.v2525\html_output'
csv_file_outputs_dir=r'C:\Users\Saranga\Desktop\Devopedia\Work\articleRefs.v2525\csv_outputs_of_HTML3'

In [2]:
import pandas as pd
import os
import re
from tqdm import tqdm

In [3]:
df=pd.read_csv('dataset_html.csv')
df.drop(['index'],axis=1,inplace=True)

In [4]:
df

,url,fname,text,citeas,file_type,file_size,Date,Title,Authors
0,https://nordvpn.com/blog/device-fingerprinting...,6ae8650d640a72aad59f0c41ed5a0355fa3467b9af348c...,"Craig, Christina. 2017. ""Device fingerprinting...",Craig 2017,text/html; charset=UTF-8,119465,2017,Device fingerprinting: The tracking we can’t a...,"Craig, Christina"
1,http://blog.madhukaraphatak.com/introduction-t...,20d9bff90a512b60097596b39546c064fe327e918fdf97...,"Phatak, Madhukara. 2016. ""Introduction to Spar...",Phatak 2016,text/html; charset=utf-8,12982,2016,Introduction to Spark 2.0 : A Sneak Peek At Ne...,"Phatak, Madhukara"
2,https://arxiv.org/abs/1906.01733,a4eb99c70ce6a8fbb257b4b6ab1f65db8f3b6bca001806...,"Alikaniotis, Dimitrios, and Vipul Raheja. 2019...",Alikaniotis and Raheja 2019,text/html; charset=utf-8,28844,2019,The Unreasonable Effectiveness of Transformer ...,"Alikaniotis, Dimitrios, and Vipul Raheja"
3,https://github.com/sunilchomal/GECwBERT,73a15c2d42e83d407d46caccfb801fa96d137aebe86947...,"Chomal, Sunil. 2019. ""GECwBERT."" sunilchomal/G...",Chomal 2019,text/html; charset=utf-8,186044,2019,GECwBERT.,"Chomal, Sunil"
4,https://tools.ietf.org/html/rfc761,363093d56adefd47bec82072842d279a0fbaccc5b075a8...,"Postel, Jon, ed. 1980b. ""Transmission Control ...",Postel 1980b,text/html; charset=UTF-8,190151,1980,Transmission Control Protocol.,"Postel, Jon, ed"
...,...,...,...,...,...,...,...,...,...
6511,https://medium.freecodecamp.org/when-should-i-...,75a131fb95733d540b7e10b78bfb664dcf0aac7a562662...,"Ewerlöf, Alex. 2016. ""When should I use TypeSc...",Ewerlöf 2016,text/html; charset=utf-8,50234,2016,When should I use TypeScript?,"Ewerlöf, Alex"
6512,https://www.tensorflow.org/guide/eager,7d6f9dd207e5df3ca527bc80d46654eb68f528d73ad831...,"TensorFlow. 2018h. ""Eager Execution."" TensorFl...",TensorFlow 2018h,text/html; charset=utf-8,124581,2018,Eager Execution.,TensorFlow
6513,https://www.oshwa.org/research/brief-history-o...,282535783fc820a98450c86dbb468f1d4ab4d469a42b8d...,"OSHWA. 2013. ""Brief History of Open Source Har...",OSHWA 2013,text/html; charset=UTF-8,81775,2013,Brief History of Open Source Hardware Organiza...,OSHWA
6514,https://medium.com/google-developers/instant-l...,ecb75f75b2275bd42ddf9fcff9dfdc3bcbbfaa89578005...,"Osmani, Addy and Matt Gaunt. 2015. ""Instant Lo...",Osmani and Gaunt 2015,text/html; charset=utf-8,301171,2015,Instant Loading Web Apps With An Application S...,"Osmani, Addy and Matt Gaunt"


In [5]:
utf=['UTF-8','utf-8','"UTF-8"','utf8',"utf-8"]

ignore_signs=['!','/']

In [10]:
error_count=0
error_items=[]
tag_special_char_check= re.compile('[^a-zA-Z0-9]')
text_special_char_check=re.compile('[^a-zA-Z0-9"]')

# lol=0

for index in tqdm(df.iterrows()):
    
#     if lol==10:
#         break
#     lol+=1
    
    tag_list=[]
    text_list=[]
    

    try:
    
        file_type=index[1]['file_type']
        file_name=index[1]['fname']
        file_path=os.path.join(html_files_dir,file_name)



        try:
            with open(file_path, encoding='utf8') as f:
                file=f.read()
                
        except:
            with open(file_path) as f:             #files which can be read using default encoding
                file=f.read()
                
        
        for i in re.finditer('<(?!\/|script|meta|button|link|style|li|nav|body|figure|code).*?>(?!<)(.+?)<',file):
            # Ignore mentioned tags
            
            tag=None
            
            tag=re.search(r'<(?!\/|script).*?>',i.group())               #tag part of the tag-text
            tag=re.search(r'([^\s]+)',tag.group())   #Word before first space is encountered (we dont need tag attributes)
            tag=re.sub(r'[<>]','',tag.group())      #remove angular brackets, and extract only tag name

            text=re.search(r'>(?!<)(.+?)<',i.group())       #Text part of the tag-text
            text=re.sub(r'[<>]','',text.group())          #Remove angular brackets, and extract only text
            
            
            if tag:
                
                if tag_special_char_check.search(tag) is not None:       #exclude tags having special characters
                    continue
                
                if len(tag)>50 or tag.isdigit():    #exclude tags having len >50 OR if tagname is only digits (invalid tag)
                    continue
                    
                if text=='':                      #exclude empty texts
                    continue
                    
                if text_special_char_check.search(text[0]) is not None: #exclude texts starting with a special char (except ")") 
                    continue
                    
                if len(text)>500 or text.isspace():  #exclude texts with len>500 OR if text is only spaces, newlines, tabs etc
                    continue
                    
                if text[0]==' ' and text_special_char_check.search(text[1]) is not None: #some texts start with a space
                    continue
                    
                if text[0]=='"' and len(re.findall(r'"',text))==2:            # tags having 2 double quotes 
                    text_within_quotes=re.search(r'"([^"]*)"', text).group()  #interested in contained text
                    
                    if not text_within_quotes[1:-1].isspace():               # excluding empty OR irrelevant strings within " " 
                        text=text_within_quotes[1:-1]                        #Text extracted without the " "
                    else:
                        continue                                            
                    
                    
                temp=re.sub('[^a-zA-Z0-9]', '', text)   #Some texts are =>                                   21.23.43
                if temp.isdigit():
                    if len(temp)<4:         #Maybe year, if less than 3 digits, exclude (not a very clean condition, I admit)
                        continue
                        
                    
                
                text=re.sub('&nbsp','',text)    #removing &nbsp
                tag_list.append(tag)
                text_list.append(text)

        n_rows=len(tag_list)
        
        RefString=[index[1].text]*n_rows
        citeas=[index[1].citeas]*n_rows
        YoP=[index[1].Date]*n_rows
        Title=[index[1].Title]*n_rows
        Author=[index[1].Authors]*n_rows
        html_fname=[index[1].fname]*n_rows
        url=[index[1].url]*n_rows

        dataframe=pd.DataFrame(list(zip(tag_list,text_list,RefString,citeas,YoP,
                                       Title,Author,html_fname,url)),
                               columns=['Tag','Text','RefString','CiteAs','YoPublishing',
                                        'Title','Author','HTML_fname','URL'])


        csv_file=os.path.join(csv_file_outputs_dir,file_name+'.csv')

        dataframe.to_csv(csv_file,index=False)
        
    except:
        error_count+=1
        error_items.append(index)
        print('Error:'+str(index[1].fname))
        pass


419it [00:13, 17.28it/s]

Error:0e1a67b25e391731b93e2ad12985395c2ff65dc58669cfda8d2f9a460b693ae0.htm


518it [00:16, 54.12it/s]

Error:fb64b9b6dc6a507b292702202601d216a737a3110e266effba5f17ef9512094e.htm


2988it [08:04,  8.47it/s]

Error:303aef879d85d6b36e241b09e51c98c179cfecbac6d7a163e416ea36caac6233.htm


6516it [19:09,  5.67it/s]
